In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
import os
import acquire
import prepare
import env

In [2]:
# Getting conncection to mySQL database, and acquiring data
def get_connection(db, user=env.user, host=env.host, password=env.password):
    '''
    This function gets conncection to mySQL database
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

# Loading raw data from Curriculum_Log database
def new_data():
    '''
    This function reads the curriculum data from the mySQL database into a df.
    '''
    # Create SQL query.
    sql_query = '''
    SELECT 
        date,
        time,
        ip,
        path,
        user_id,
        cohort_id,
        name as cohort_name,
        slack,
        start_date,
        end_date,
        program_id
    FROM
        curriculum_logs.logs
    join
        curriculum_logs.cohorts on cohort_id = id
    ;
    '''
    # Read in DataFrame from Codeup db.
    df = pd.read_sql(sql_query, get_connection('curriculum_logs'))
    
    return df

def get_data():
    '''
    This function reads in curriculum data from curriculum_log database, writes data to
    a csv file if a local file does not exist, and returns a df.
    '''
    if os.path.isfile('curriculum.csv'):
        
        # If csv file exists, read in data from csv file.
        df = pd.read_csv('curriculum.csv', index_col=0)
        
    else:
        
        # Read fresh data from db into a DataFrame.
        df = new_data()
        
        # Write DataFrame to a csv file.
        df.to_csv('curriculum.csv')
        
    return df

In [3]:
def prep_data(df):
    '''
    This function takes in a messy dataframe and return the cleaned verison of dataframe.
    Detial steps are in code comment below.
    '''
    # Set date columns as datatime as index
    df.date = pd.to_datetime(df.date)
    df = df.set_index(df.date)

    # Drop null values
    df = df.dropna()

    # Encode program_id
    df['program'] = df.program_id.map({1: 'Full Stack PHP', 2: 'Full Stack Java', 3: 'Data Science', 4: 'Front End'})

    return df

In [4]:
df = acquire.get_data()
df = prepare.prep_data(df)

In [5]:
df.head()

,date,time,ip,path,user_id,cohort_id,cohort_name,slack,start_date,end_date,program_id,program
date,,,,,,,,,,,,
2018-01-26,2018-01-26,09:55:03,97.105.19.61,/,1,8.0,Hampton,#hampton,2015-09-22,2016-02-06,1,Full Stack PHP
2018-01-26,2018-01-26,09:56:02,97.105.19.61,java-ii,1,8.0,Hampton,#hampton,2015-09-22,2016-02-06,1,Full Stack PHP
2018-01-26,2018-01-26,09:56:05,97.105.19.61,java-ii/object-oriented-programming,1,8.0,Hampton,#hampton,2015-09-22,2016-02-06,1,Full Stack PHP
2018-01-26,2018-01-26,09:56:06,97.105.19.61,slides/object_oriented_programming,1,8.0,Hampton,#hampton,2015-09-22,2016-02-06,1,Full Stack PHP
2018-01-26,2018-01-26,09:56:24,97.105.19.61,javascript-i/conditionals,2,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2,Full Stack Java


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 847329 entries, 2018-01-26 to 2021-04-21
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   date         847329 non-null  datetime64[ns]
 1   time         847329 non-null  object        
 2   ip           847329 non-null  object        
 3   path         847329 non-null  object        
 4   user_id      847329 non-null  int64         
 5   cohort_id    847329 non-null  float64       
 6   cohort_name  847329 non-null  object        
 7   slack        847329 non-null  object        
 8   start_date   847329 non-null  object        
 9   end_date     847329 non-null  object        
 10  program_id   847329 non-null  int64         
 11  program      847329 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(8)
memory usage: 84.0+ MB


In [7]:
# Drop deleted_at column


#df = df.drop(columns= 'deleted_at', inplace = True)
#df.head()

## Detecting Anomalies in Discrete Variables

1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?

---

#### Okay so how do I best approach this problem?

1. Create and separate the datasets of Web Development and Data Science programs.
    - 1 == WD
    - 2 == WD
    - 3 == DS
    - 4 == WD
    
2. Remove the popular logistic pages in the curriculum. I.E. '/' = main page, 'toc' = Table of Contents, et.c
3. Use .value_counts() on path to see the most accessed pages.
4. Store the top 10 page names with page counts as a variable.

In [8]:
df.path.value_counts()

/                                                               45854
javascript-i                                                    18203
toc                                                             17591
search/search_index.json                                        17534
java-iii                                                        13166
                                                                ...  
content/examples/javascript/primitive-types.html                    1
content/examples/javascript/conditionals.html                       1
2-storytelling/1-overview/www.qlik.com                              1
syntax-types-and-variables                                          1
appendix/professional-development/post-interview-review-form        1
Name: path, Length: 2224, dtype: int64

In [9]:
df.path.value_counts(normalize=True).head(20)

/                                                                            0.054116
javascript-i                                                                 0.021483
toc                                                                          0.020761
search/search_index.json                                                     0.020693
java-iii                                                                     0.015538
html-css                                                                     0.015492
java-ii                                                                      0.014371
spring                                                                       0.014024
jquery                                                                       0.013030
mysql                                                                        0.012523
java-i                                                                       0.012353
javascript-ii                                         

In [10]:
# how many unique paths we have in the dataset?
df.path.nunique()

2224

In [11]:
df.nunique()

date            1182
time           72331
ip              5200
path            2224
user_id          911
cohort_id         47
cohort_name       47
slack             46
start_date        44
end_date          45
program_id         4
program            4
dtype: int64

### Top 10 accessed pages across programs

In [12]:
df.loc[df.program_id==1].path.value_counts().nlargest(10)

/                1681
index.html       1011
javascript-i      736
html-css          542
spring            501
java-iii          479
java-ii           454
java-i            444
javascript-ii     429
appendix          409
Name: path, dtype: int64

In [13]:
df.loc[df.program_id==2].path.value_counts().nlargest(10)

/                           35814
javascript-i                17457
toc                         17428
search/search_index.json    15212
java-iii                    12683
html-css                    12569
java-ii                     11719
spring                      11376
jquery                      10693
mysql                       10318
Name: path, dtype: int64

In [14]:
df.loc[df.program_id==3].path.value_counts().nlargest(10)

/                                           8358
search/search_index.json                    2203
classification/overview                     1785
1-fundamentals/modern-data-scientist.jpg    1655
1-fundamentals/AI-ML-DL-timeline.jpg        1651
1-fundamentals/1.1-intro-to-data-science    1633
classification/scale_features_or_not.svg    1590
fundamentals/AI-ML-DL-timeline.jpg          1443
fundamentals/modern-data-scientist.jpg      1438
sql/mysql-overview                          1424
Name: path, dtype: int64

In [15]:
df.loc[df.program_id==4].path.value_counts().nlargest(10)

content/html-css                               2
/                                              1
content/html-css/gitbook/images/favicon.ico    1
content/html-css/introduction.html             1
Name: path, dtype: int64

## Web Development up first

In [16]:
# Remove staff from the analysis.
df.loc[~(df.cohort_name == 'Staff')]

# Step 1.
# Web Devlopment Cohorts
wd_cohorts = df.loc[df.program_id.isin([1, 2, 4])]

# Data Science Cohorts
ds_cohorts = df.loc[df.program_id == 3]

In [17]:
# Step 2.
# Create a variable to store pages to filter out.
remove_pages = [ 'index.html', 'search/search_index.json', 'mkdocs/search_index.json','/', 'toc', 'appendix',]

# Clean page_viewed to remove logistic pages: Searches, Table of Contents, Appendix
wd_cohorts = wd_cohorts.loc[~wd_cohorts.path.isin(remove_pages)]

# Replace content main page with empty string.
wd_cohorts.page_viewed = wd_cohorts.path.replace('content/', '', regex=True)

/var/folders/xr/rf33rdr91vd7xt659vgf8qkm0000gn/T/ipykernel_4575/1308730922.py:9: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  wd_cohorts.page_viewed = wd_cohorts.path.replace('content/', '', regex=True)


In [26]:
wd_cohorts.head()

,date,time,ip,path,user_id,cohort_id,cohort_name,slack,start_date,end_date,program_id,program
date,,,,,,,,,,,,
2018-01-26,2018-01-26,09:56:02,97.105.19.61,java-ii,1,8.0,Hampton,#hampton,2015-09-22,2016-02-06,1,Full Stack PHP
2018-01-26,2018-01-26,09:56:05,97.105.19.61,java-ii/object-oriented-programming,1,8.0,Hampton,#hampton,2015-09-22,2016-02-06,1,Full Stack PHP
2018-01-26,2018-01-26,09:56:06,97.105.19.61,slides/object_oriented_programming,1,8.0,Hampton,#hampton,2015-09-22,2016-02-06,1,Full Stack PHP
2018-01-26,2018-01-26,09:56:24,97.105.19.61,javascript-i/conditionals,2,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2,Full Stack Java
2018-01-26,2018-01-26,09:56:41,97.105.19.61,javascript-i/loops,2,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2,Full Stack Java


In [19]:
# Step 3 `.value_counts()`
wd_top_10_pages = wd_cohorts.path.value_counts().nlargest(10)
print(wd_top_10_pages)
wd_top_10_pages = wd_top_10_pages.index.to_list()

javascript-i                                                                 18193
java-iii                                                                     13162
html-css                                                                     13111
java-ii                                                                      12173
spring                                                                       11877
jquery                                                                       11037
mysql                                                                        10602
java-i                                                                       10460
javascript-ii                                                                10290
javascript-i/introduction/working-with-data-types-operators-and-variables     7326
Name: path, dtype: int64


## Data Science up next

In [27]:
# Step 2.
# Create a variable to store pages to filter out.
remove_pages = ['/', 'toc', 'appendix', 'index.html', 'search/search_index.json', 'mkdocs/search_index.json']

# Clean page_viewed to remove logistic pages: Searches, Table of Contents, Appendix
ds_cohorts = ds_cohorts.loc[~ds_cohorts.path.isin(remove_pages)]

# Replace content main page with empty string.
ds_cohorts.path = ds_cohorts.path.replace('content/', '', regex=True)

In [28]:
ds_cohorts.head()

,date,time,ip,path,user_id,cohort_id,cohort_name,slack,start_date,end_date,program_id,program
date,,,,,,,,,,,,
2019-08-20,2019-08-20,09:40:28,97.105.19.58,3-sql/1-mysql-overview,470,34.0,Bayes,#,2019-08-19,2020-01-30,3,Data Science
2019-08-20,2019-08-20,09:40:30,97.105.19.58,2-storytelling/bad-charts,467,34.0,Bayes,#,2019-08-19,2020-01-30,3,Data Science
2019-08-20,2019-08-20,09:40:30,97.105.19.58,2-storytelling/misleading1_baseball.jpg,467,34.0,Bayes,#,2019-08-19,2020-01-30,3,Data Science
2019-08-20,2019-08-20,09:40:30,97.105.19.58,2-storytelling/misleading1_fox.jpg,467,34.0,Bayes,#,2019-08-19,2020-01-30,3,Data Science
2019-08-20,2019-08-20,09:40:30,97.105.19.58,2-storytelling/misleading3_deaths.jpg,467,34.0,Bayes,#,2019-08-19,2020-01-30,3,Data Science


In [29]:
# Step 3 `.value_counts()`
ds_top_10_pages = ds_cohorts.path.value_counts().nlargest(10)
print(ds_top_10_pages)
ds_top_10_pages = ds_top_10_pages.index.to_list()

classification/overview                     1785
1-fundamentals/modern-data-scientist.jpg    1655
1-fundamentals/AI-ML-DL-timeline.jpg        1651
1-fundamentals/1.1-intro-to-data-science    1633
classification/scale_features_or_not.svg    1590
fundamentals/AI-ML-DL-timeline.jpg          1443
fundamentals/modern-data-scientist.jpg      1438
sql/mysql-overview                          1424
fundamentals/intro-to-data-science          1413
6-regression/1-overview                     1124
Name: path, dtype: int64


# Takeaways for Question 1.

- Data Science cohorts access <b>classification/overview</b>  the most.
- Web Development cohorts access <b>javascript-i</b> the most.

# Question 7. Which lessons are least accessed?

In [53]:
wd_bottom_10_pages = wd_cohorts.path.value_counts().nsmallest(10)
print(wd_bottom_10_pages)
wd_bottom_10_pages = wd_bottom_10_pages.index.to_list()

javascript-i/introduction/working-with-data-types-operators-and-variables/www.google.com                                                                          1
timeline/java-i                                                                                                                                                   1
teams/13                                                                                                                                                          1
elements/html                                                                                                                                                     1
javascript-i/introduction/working-with-data-types-operators-and-variables/600%20Navarro%20St.,%20San%20Antonio,%20TX%2078205,%20United%20States%20of%20America    1
capstone/index.md                                                                                                                                                 1
hfdgafdja       

In [64]:
ds_bottom_10_pages = ds_cohorts.path.value_counts().nsmallest(100)
print(ds_bottom_10_pages)
ds_bottom_10_pages = ds_bottom_10_pages.index.to_list()

classification/explore-old          1
where                               1
Index.html                          1
python/introduction-to-python.md    1
sql                                 1
                                   ..
A-clustering/project                1
b-clustering/project                1
misleading1_baseball.jpg            1
misleading1_fox.jpg                 1
python/custom-sorting-functions     1
Name: path, Length: 100, dtype: int64


## Takeaways:
- For Web Development, there are 400+ pages that were accessed only once.
- For Data Science, there are 100+ pages that were accessed only once.
- Some commonalities are that many of the pages are introduction pages with no useful information.

# Question 3: Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

In [236]:
#Create an alumni column 
df['alumni']= np.where((df['date']<df['end_date']),'alumni','current')

In [259]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 681164 entries, 2018-01-26 to 2021-04-21
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   date         681164 non-null  datetime64[ns]
 1   time         681164 non-null  object        
 2   ip           681164 non-null  object        
 3   path         681164 non-null  object        
 4   user_id      681164 non-null  int64         
 5   cohort_id    681164 non-null  float64       
 6   cohort_name  681164 non-null  object        
 7   slack        681164 non-null  object        
 8   start_date   681164 non-null  object        
 9   end_date     681164 non-null  object        
 10  program_id   681164 non-null  int64         
 11  program      681164 non-null  object        
 12  alumni       681164 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(9)
memory usage: 88.9+ MB


In [238]:
#create a data subset
current_students = df.loc[df["alumni"] != 'alumni']

In [239]:
current_students.shape

(99217, 13)

In [240]:
df.cohort_name.nunique()

45

In [241]:
df.user_id.nunique()

873

In [255]:
# Create a subset of students that only accessed the curriculum once.

#current_students_upper_ids = current_students.groupby('user_id').path.count()[current_students.groupby('user_id').path.count() > 800].index.to_list()
current_students_lower_ids = pd.DataFrame(current_students.groupby('user_id').path.count())
#[current_students.groupby('user_id').path.count() <=1 ].index.to_list()

In [257]:
# 12 students accessed the curriculum only once.
current_students_lower_ids[current_students_lower_ids.path <= 1]

,path
user_id,
81,1
97,1
212,1
240,1
250,1
348,1
394,1
415,1
519,1


In [244]:
# create a loop. create a df showing that. 

In [245]:
#current_students_low_access = df.loc[df.user_id.isin[[81, 97, 212, 240, 250, 348, 394, 415, 519, 701, 755, 857]]]

In [258]:
df.loc[df.user_id==755].path

date
2020-09-21                                 javascript-i
2020-09-21                    javascript-i/introduction
2020-09-21    javascript-i/introduction/primitive-types
2020-09-22                            javascript-ii/es6
2020-09-22                                 javascript-i
                                ...                    
2021-03-23                                       spring
2021-03-23             spring/fundamentals/repositories
2021-03-23                                     java-iii
2021-03-23                    java-iii/mvc/introduction
2021-04-04                          javascript-i/arrays
Name: path, Length: 739, dtype: object

In [247]:
df.loc[df.user_id==97].path

date
2018-02-11    javascript-i
Name: path, dtype: object

In [252]:
df.loc[df.user_id==755].path

date
2020-09-21                                 javascript-i
2020-09-21                    javascript-i/introduction
2020-09-21    javascript-i/introduction/primitive-types
2020-09-22                            javascript-ii/es6
2020-09-22                                 javascript-i
                                ...                    
2021-03-23                                       spring
2021-03-23             spring/fundamentals/repositories
2021-03-23                                     java-iii
2021-03-23                    java-iii/mvc/introduction
2021-04-04                          javascript-i/arrays
Name: path, Length: 739, dtype: object

In [249]:
current_students_lower_ids_findings = [df.loc[df.user_id==81].path, 
df.loc[df.user_id==97].path,
df.loc[df.user_id==212].path, 
df.loc[df.user_id==240].path,
df.loc[df.user_id==250].path,
df.loc[df.user_id==348].path,
df.loc[df.user_id==394].path,
df.loc[df.user_id==415].path,
df.loc[df.user_id==519].path,
df.loc[df.user_id==701].path,
df.loc[df.user_id==755].path,
df.loc[df.user_id==857].path]
                                       

In [250]:
df.loc[df.user_id==755].path

date
2020-09-21                                 javascript-i
2020-09-21                    javascript-i/introduction
2020-09-21    javascript-i/introduction/primitive-types
2020-09-22                            javascript-ii/es6
2020-09-22                                 javascript-i
                                ...                    
2021-03-23                                       spring
2021-03-23             spring/fundamentals/repositories
2021-03-23                                     java-iii
2021-03-23                    java-iii/mvc/introduction
2021-04-04                          javascript-i/arrays
Name: path, Length: 739, dtype: object

In [184]:
current_students_lower_ids_findings = [df.loc[df.user_id==81].path, df.loc[df.user_id==97].path,

In [187]:
348, 857, 212, 97, 81

date
2020-09-21                                 javascript-i
2020-09-21                    javascript-i/introduction
2020-09-21    javascript-i/introduction/primitive-types
2020-09-22                            javascript-ii/es6
2020-09-22                                 javascript-i
                                ...                    
2021-03-23                                       spring
2021-03-23             spring/fundamentals/repositories
2021-03-23                                     java-iii
2021-03-23                    java-iii/mvc/introduction
2021-04-04                          javascript-i/arrays
Name: path, Length: 739, dtype: object